# Data Visualization Module
The purpose of this notebook is to act as a module that can be imported into the Master Notebook to visualize various data.

## Fine Particulate Matter (PM2.5) Time Series
The following function plots PM2.5 concentration as either $\mu$g/m$^3$ or AQI. The background is formatted such that the breakpoints for AQI are included. Breakpoints for AQI can be found [here](http://aqicn.org/faq/2013-09-09/revised-pm25-aqi-breakpoints/). 

In [2]:
def getFinePMTimeSeries(x, y, ylabel, day_no=0, figure_size=(16,8)):
    '''
    Inputs:
        - x: numpy DateTime array holding the independent variable data
        - y: numpy float array holding the PM2.5 concentration or AQI
        - ylabel: string representing the y-axis label 
        - day_no: integer for what day in the series it is
        - figure_size: a tuple holding the figure size; default is (16,8)
    Returns a time series plot of PM2.5 Concentration or AQI with AQI breakpoints included as background
    '''
    # Plotting
    fig, ax = plt.subplots(figsize = figure_size)
    ax.plot(x,y,color='black',linewidth=3,label='PM2.5')

    ## Formatting x-axis
    ax.set_xlim([x[0],x[-1]])
    ### Formatting datetime labels
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d %H:%M'))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M'))
    ax.xaxis.set_minor_locator(mdates.HourLocator(byhour=range(2,24,2)))
    ax.set_xlabel('Time')
    ax.xaxis.set_tick_params(rotation=-30)
    
    ## Formatting y-axis
    ax.set_ylim([0,500])
    ax.set_ylabel(ylabel)
    
    ## Formatting remaining aspects of figure
    ax.set_title('Day ' + str(day_no) + ': ' + str(x[2].date()))
    ### AQI breakpoints
    bp_c_start = [0,12,35.5,55.5,150.5,250.5]
    bp_c_end = [12,35.4,55.4,150.4,250.4,500]
    bp_aqi_start = [0,51,101,151,201,301]
    bp_aqi_end = [50,100,150,200,300,500]
    if ylabel == 'Air Quality Index':
        bp_start = bp_aqi_start
        bp_end = bp_aqi_end
    else:
        bp_start = bp_c_start
        bp_end = bp_c_end
    ax.axhspan(bp_start[0],bp_end[0],color='green',alpha=0.3,label='Good')
    ax.axhspan(bp_start[1],bp_end[1],color='yellow',alpha=0.3,label='Moderate')
    ax.axhspan(bp_start[2],bp_end[2],color='orange',alpha=0.3,label='Unhealthy for Children/Elderly')
    ax.axhspan(bp_start[3],bp_end[3],color='red',alpha=0.3,label='Unhealthy')
    ax.axhspan(bp_start[4],bp_end[4],color='purple',alpha=0.3,label='Very Unhealthy')
    ax.axhspan(bp_start[5],bp_end[5],color='maroon',alpha=0.3,label='Hazardous')
    ax.legend()